<a href="https://colab.research.google.com/github/bluebynick/CEE-9730-Data-Science-for-Geotechnical-Engineers/blob/main/assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Cell 1: install and imports, mount Drive
#!pip install -q tensorflow==2.15.0  # or a current TF2.x
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, glob, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
## from tensorflow import keras
## from tensorflow import layers
from sklearn.metrics import confusion_matrix
print("TensorFlow version:", tf.__version__)

# Cell 2: user settings - EDIT THESE
leftImg8bit = os.path.join('/content/drive/MyDrive/leftImg8bit_trainvaltest', 'leftImg8bit')
gtCoarse = os.path.join('/content/drive/MyDrive/gtCoarse', 'gtCoarse')

IMAGE_SIZE = (256, 512)   # (height, width) - reduce for speed.
BATCH_SIZE = 8
EPOCHS = 20               # how many epochs per LR run (reduce for quick debug)
N_CLASSES = 12            # number of target classes in assignment (we map Cityscapes trainIds -> these)
IGNORE_LABEL = 255

# For debugging fast: limit number of files
DEBUG_LIMIT_TRAIN = 200   # set None to use all
DEBUG_LIMIT_VAL = 50

print("Paths set. leftImg8bit exists?", os.path.exists(leftImg8bit))
print("gtCoarse exists?", os.path.exists(gtCoarse))

# Cell 4: helper to find image/label pairs
def find_image_label_pairs(split='train'):
    img_pattern = os.path.join(leftImg8bit, split, '*', '*_leftImg8bit.png')
    # two possible coarse label naming conventions - try both
    label_pattern1 = os.path.join(gtCoarse, split, '*', '*_gtCoarse_labelTrainIds.png')
    label_pattern2 = os.path.join(gtCoarse, split, '*', '*_gtCoarse_labelIds.png')
    imgs = sorted(glob.glob(img_pattern))
    labels = {}
    for p in glob.glob(label_pattern1):
        labels[Path(p).stem.replace('_gtCoarse_labelTrainIds','')] = p
    for p in glob.glob(label_pattern2):
        labels[Path(p).stem.replace('_gtCoarse_labelIds','')] = p
    pairs = []
    for img in imgs:
        stem = Path(img).stem.replace('_leftImg8bit','')
        lab = labels.get(stem)
        if lab:
            pairs.append((img, lab))
    return pairs

train_pairs = find_image_label_pairs('train')
val_pairs = find_image_label_pairs('val')
print("Found pairs:", len(train_pairs), "train |", len(val_pairs), "val")

# optionally reduce for a quick run
if DEBUG_LIMIT_TRAIN:
    train_pairs = train_pairs[:DEBUG_LIMIT_TRAIN]
if DEBUG_LIMIT_VAL:
    val_pairs = val_pairs[:DEBUG_LIMIT_VAL]
print("Using pairs:", len(train_pairs), "train |", len(val_pairs), "val")


Mounted at /content/drive
TensorFlow version: 2.19.0
Paths set. leftImg8bit exists? True
gtCoarse exists? True
unzipped
Found pairs: 2500 train | 500 val
Using pairs: 200 train | 50 val
